# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234395064604                   -0.50    8.0         
  2   -7.249962260648       -1.81       -1.39    1.0   5.14ms
  3   -7.251209270528       -2.90       -1.98    2.0   6.26ms
  4   -7.251272807033       -4.20       -2.16    2.0   6.36ms
  5   -7.251324122994       -4.29       -2.57    1.0   5.29ms
  6   -7.251336262495       -4.92       -2.95    1.0   5.32ms
  7   -7.251338644646       -5.62       -3.58    1.0   5.35ms
  8   -7.251338794063       -6.83       -4.17    2.0   6.55ms
  9   -7.251338798146       -8.39       -4.70    2.0   6.61ms
 10   -7.251338798585       -9.36       -5.01    2.0   6.84ms
 11   -7.251338798697       -9.95       -5.67    2.0   6.60ms
 12   -7.251338798703      -11.19       -5.97    2.0   6.90ms
 13   -7.251338798704      -11.95       -6.40    1.0   5.84ms
 14   -7.251338798705      -12.73       -6.85    2.0   6.91ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04376728086640632
[ Info: Arnoldi iteration step 2: normres = 0.42025536684038034
[ Info: Arnoldi iteration step 3: normres = 0.8228270881445805
[ Info: Arnoldi iteration step 4: normres = 0.41869027051783125
[ Info: Arnoldi iteration step 5: normres = 0.5325774653022357
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.85e-02, 6.64e-02, 4.41e-01, 2.72e-01, 3.11e-02)
[ Info: Arnoldi iteration step 6: normres = 0.30442514891072614
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.43e-02, 8.68e-02, 2.52e-01, 1.14e-01, 6.97e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08784759665633819
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.07e-03, 9.07e-03, 1.97e-02, 5.54e-02, 5.70e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09997860186168549
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.63